In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import xarray as xr
from math import atan2, log
#read in email
text_file = open('C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/baja/docs/email_for_file.txt', "r")
email_str = text_file.readlines()


In [2]:
#definitions and things you might have to set differently for each file
itow_mask1=45 #see code just a bit below for figure to determine where to set flags here
itow_mask2=-110 #see code just a bit below for figure to determine where to set flags here
ISDP = 'Saildrone'
SST_type = 'SSTdepth'
Annex_version = '01.1'
File_version = '01.0'
astr_platform='SD1002'
astr_title = 'Data from Saildrone cruise from SF to Guadalupe Island April-June 2018'
astr_uuid = '0f410de6-4ba5-4f79-af20-8a57a445f454'
#input filename
dir_in='f:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/2018/'
dir_out='f:/data/cruise_data/saildrone/baja-2018/other_format_shared_data/'
droplist=['WWND_STDDEV', 'CHLOR_MEAN','RH_MEAN','WWND_MEAN','O2_CONC_STDDEV','CDOM_STDDEV',
                          'TEMP_O2_STDDEV','BARO_PRES_MEAN','TEMP_O2_MEAN','SAL_STDDEV','TEMP_AIR_MEAN',
                          'CDOM_MEAN','SAL_MEAN','O2_SAT_MEAN','CHLOR_STDDEV', 'COND_STDDEV', 'COND_MEAN',
                          'BKSCT_RED_MEAN', 'TEMP_IR_MEAN', 'O2_SAT_STDDEV','O2_CONC_MEAN', 'TEMP_AIR_STDDEV', 
                          'BARO_PRES_STDDEV', 'TEMP_IR_STDDEV', 'VWND_STDDEV','RH_STDDEV', 'GUST_WND_STDDEV', 'GUST_WND_MEAN',
                          'BKSCT_RED_STDDEV', 'UWND_STDDEV','HDG_WING','WING_ANGLE']


In [3]:
#read data back in, into two arrays one with time encoding and one without
adir='F:/data/cruise_data/saildrone/baja-2018/'
#adir='C:/Users/gentemann/Desktop/data/'
filename_usv=adir+'saildrone-gen_5-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v2.1535585233403.nc'
#fname=adir+'saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1_with_loggers.nc'
with xr.open_dataset(filename_usv,drop_variables=droplist) as ds:
    lons_usv=ds.longitude[0,:]
    lats_usv=ds.latitude[0,:]
    time_usv=ds.time[0,:]
with xr.open_dataset(filename_usv,drop_variables=droplist,decode_times=False) as ds_raw:
    time_usv_raw=ds_raw.time[0,:]

In [4]:
mint=time_usv.min().data
maxt=time_usv.max().data
ds.attrs['time_coverage_start']=str(np.datetime64(mint,'ms'))+'Z'
ds.attrs['time_coverage_end']=str(np.datetime64(maxt,'ms'))+'Z'
ilen=(len(ds.latitude['obs']))
#dates_usv64=dataset_decodetime.TIME[0,:].values
#time_usv_pd=pd.to_datetime(time_usv, unit='ns')
print(ilen)


86839


In [5]:
#calculate the average distance between observations as the spatial resolution global attribute
#import math
#from math import cos
# approximate radius of earth in km
R = 6373.0 #km
lat1 = np.deg2rad(lats_usv[1:ilen])
lon1 = np.deg2rad(lons_usv[1:ilen])
lat2 = np.deg2rad(lats_usv[0:ilen-1])
lon2 = np.deg2rad(lons_usv[0:ilen-1])
dlon = lon2 - lon1
dlat = lat2 - lat1
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(a**.5, (1 - a)**.5)
distance = R * c

In [6]:
#add global attributes that are missing
#some of these will need to be changed for new cruises
dataset=ds
gattrs = dataset.attrs.copy()
gattrs['title'] = astr_title
gattrs['summary'] = 'none'
gattrs['references'] = 'none'
gattrs['institution'] = 'Saildrone'
gattrs['history'] = 'Saildrone 6-hourly v1 files were used to create this file'
gattrs['comment'] = 'none'
gattrs['license'] = 'CC-BY-NC' #'free and open'
gattrs['id'] = 'SSTdepth'
gattrs['naming_authority'] = 'org.shipborne-radiometer'
gattrs['product_version'] = '1.0'
gattrs['uuid'] = astr_uuid 
gattrs['l3r_version_id'] = '1.1' 
gattrs['netcdf_version_id'] = '4.6.1'
gattrs['date_created'] = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ") #yyyy-mm-ddThh:mm:ssZ
gattrs['file_quality_level'] = 3
gattrs['spatial_resolution'] = str(distance.mean().data*1000)+' m'
gattrs['start_time'] = ds.time_coverage_start
gattrs['stop_time'] = ds.time_coverage_end
gattrs['northernmost_latitude'] = lats_usv.max().data
gattrs['geospatial_lat_max'] = lats_usv.max().data
gattrs['southernmost_latitude'] = lats_usv.min().data
gattrs['geospatial_lat_min'] = lats_usv.min().data
gattrs['easternmost_longitude'] = lons_usv.max().data
gattrs['geospatial_lon_max'] = lons_usv.max().data
gattrs['westernmost_longitude'] = lons_usv.min().data
gattrs['geospatial_lon_min'] = lons_usv.min().data
gattrs['geospatial_lat_units'] = 'degrees_north'
gattrs['geospatial_lon_units'] = 'degrees_east'
gattrs['source'] = 'SSTdepth, wind_speed'
gattrs['platform'] = astr_platform
gattrs['sensor'] = str(dataset.TEMP_CTD_MEAN[0,0].device_name + ', ' + \
            dataset.UWND_MEAN[0,0].device_name)
#Teledyne CTD (2632)_Teledyne CTD (2632)_2632, Gill Anemometer (172107)_Gill Anemometer (172107)_172107
#print(gattrs['sensor'])
gattrs['metadata_link'] = 'TBD'
gattrs['keywords'] = 'Oceans > Ocean Temperature > Sea Surface Temperature'
gattrs['keywords_vocabulary'] = 'NASA Global Change Master Directory (GCMD) Science Keywords'
gattrs['acknowledgment'] = 'The Schmidt Family Foundation, Saildrone, NASA Physical Oceanography'
gattrs['project'] = 'International Shipborne Radiometer Network'
gattrs['publisher_name'] = 'The ISRN Project Office'
gattrs['publisher_url'] = 'http://www.shipborne.radiometer.org'
gattrs['publisher_email'] = str(email_str[0])
#print(gattrs['publisher_email'])
gattrs['processing_level'] = '1.0'
#del dataset.attrs['nodc_template_version']

In [7]:
#dataset
#dataset.coords
#print(type(ds.data_vars))

In [8]:
# convert names
COORD_ATTR = 'lat lon time'
#COORD_ATTR = 'time'
FLOAT_FILLVALUE = np.finfo(np.float32).min
VAR_TRANSLATE = {
    'TEMP_CTD_MEAN': 'sea_water_temperature',
    'COG': 'course_over_ground',
    'latitude': 'lat',
    'longitude': 'lon',
    'HDG': 'true_bearing',
    'ROLL': 'platform_roll',
    'PITCH': 'platform_pitch',
    'SOG': 'speed_over_ground',
    'TEMP_CTD_STDDEV':'sst_total_uncertainty'
}
# copy variables from source dataset
varrays = {}
for v in dataset.data_vars:
    if v in VAR_TRANSLATE:
       # print(v)
        # set attributes
        vattrs = dataset[v].attrs.copy()
        if v not in ['latitude', 'longitude', 'time']:
            vattrs["coordinates"] = COORD_ATTR
        vattrs["_FillValue"] = FLOAT_FILLVALUE
        
        values = np.ma.fix_invalid(dataset[v].values[0,:])
        if v == 'TEMP_CTD_MEAN':
            # convert SST to Kelvin
            values += 273.15        
        # create new data array
        varrays[VAR_TRANSLATE[v]] = xr.DataArray(
            values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
            dims=['time'],
            attrs=vattrs
        )
for v in dataset.coords:
    if v in VAR_TRANSLATE:
       # print(v)
        # set attributes
        vattrs = dataset[v].attrs.copy()
        if v not in ['latitude', 'longitude', 'time']:
            vattrs["coordinates"] = COORD_ATTR
        vattrs["_FillValue"] = FLOAT_FILLVALUE
        
        values = np.ma.fix_invalid(dataset[v].values[0,:])
        if v == 'TEMP_CTD_MEAN':
            # convert SST to Kelvin
            values += 273.15        
        # create new data array
        varrays[VAR_TRANSLATE[v]] = xr.DataArray(
            values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
            dims=['time'],
            attrs=vattrs
        )

      
# 1. wind speed
vattrs = dataset.UWND_MEAN.attrs.copy()
vattrs['standard_name'] = 'wind_speed'
vattrs['long_name'] = 'wind_speed'
vattrs['valid_min'] = 0
vattrs['valid_max'] = 100
vattrs["_FillValue"] = FLOAT_FILLVALUE
vattrs['source'] = 'anemometer'
vattrs['comment'] = 'Instrument located at to of Saildrone mast at ' +\
                str(dataset.UWND_MEAN.installed_height)+' m' + '.  This was adjusted ' +\
                'to 10 m as ws_10m = ws*log(10./1e-4))/log(WS_height/1e-4'
vattrs['height'] = '10 m' #str(str(dataset.UWND_MEAN.installed_height)+' m')
vattrs["coordinates"] = COORD_ATTR
WS=(dataset['UWND_MEAN'].values[0,:]**2 + dataset['VWND_MEAN'].values[0,:]**2)**.5
WS_height=int(dataset.UWND_MEAN.installed_height)
WS_10m = (WS*log(10./1e-4))/log(WS_height/1e-4)
varrays['wind_speed'] = xr.DataArray(
    WS_10m.astype(np.float32,copy=False),
    dims=['time'],
    attrs=vattrs
    )

# 2. wind direction
vattrs = dataset.UWND_MEAN.attrs.copy()
vattrs['standard_name'] = 'wind_to_direction'
vattrs['long_name'] = 'local wind direction'
vattrs['valid_min'] = 0
vattrs['valid_max'] = 360
vattrs["_FillValue"] = FLOAT_FILLVALUE
vattrs['units'] = 'degrees'
vattrs['source'] = 'anemometer'
vattrs['height'] = str(str(dataset.UWND_MEAN.installed_height)+' m')
vattrs["coordinates"] = COORD_ATTR
WD=np.arctan2(dataset.VWND_MEAN.values[0,:], dataset.UWND_MEAN.values[0,:])*180/3.14159
WD=np.where(WD<0,WD+360,WD)
varrays['wind_direction'] = xr.DataArray(
    WD.astype(np.float32,copy=False),
    dims=['time'],
    attrs=vattrs
    )

# 3. time
vattrs = ds_raw.time.attrs.copy()
vattrs["_FillValue"] = FLOAT_FILLVALUE
values = np.ma.fix_invalid(ds_raw.time.values[0,:])
vtime = xr.DataArray(
    values.filled(FLOAT_FILLVALUE).astype(np.float32,copy=False),
    dims=['time'],
    attrs=vattrs
    )


# 4. Quality level
flag_bytes=np.byte((0,1,2,3,4,5)) #bytearray([0,1,2,3,4,5])
iqual_byte = np.ones(shape=time_usv.shape, dtype='b')*5  #change byte to b1
iqual_byte[:itow_mask1] = 2  #set at top of file from looking at data
iqual_byte[itow_mask2:] = 2
vattrs = {
    'long_name': 'measurement quality value',
    'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
    'flag_values': flag_bytes,
    '_FillValue': -128
}
varrays['quality_level'] = xr.DataArray(
    iqual_byte,
    dims=['time'],
    attrs=vattrs
    )


# create Dataset and save
l3r = xr.Dataset(
    varrays,
    coords = {
#        'time': time_usv
        'time': vtime
    },
    attrs=gattrs
)

# embellishments
l3r.sea_water_temperature.attrs['valid_min']=260.0
l3r.sea_water_temperature.attrs['valid_max']=320.0
l3r.sea_water_temperature.attrs['units']='kelvin'
l3r.sea_water_temperature.attrs['long_name']='sea surface depth temperature at 0.6m'
l3r.time.attrs['standard_name']='time'
l3r.time.attrs['long_name']='time'
l3r.lon.attrs['standard_name']='longitude'
l3r.lon.attrs['long_name']='longitude'
l3r.lat.attrs['long_name']='latitude'
l3r.lat.attrs['standard_name']='latitude'
l3r.true_bearing.attrs['long_name']='platform true bearing'
l3r.true_bearing.attrs['standard_name']='platform_orientation'
l3r.speed_over_ground.attrs['long_name']='platform speed over ground'
l3r.sst_total_uncertainty.attrs['standard_name']='sea_water_temperature standard error'
l3r.sst_total_uncertainty.attrs['long_name']=' sea water temperature total uncertainty'
l3r.sst_total_uncertainty.attrs['valid_min']=0.0
l3r.sst_total_uncertainty.attrs['valid_max']=2.0
l3r.sst_total_uncertainty.attrs['units']='kelvin'

indicative_date_time=pd.to_datetime(str(time_usv[0].data)).strftime("%Y%m%d%H%M%S")
Product_string = str(ds.TEMP_CTD_MEAN.vendor_name) + '_' + str(ds.TEMP_CTD_MEAN.serial_number)
filename_L3R = dir_out + indicative_date_time + \
    '-' + ISDP + '-' + 'L3R' + '-' + SST_type + '-' +Product_string+ '-v' +Annex_version+ '-fv' +File_version+ '.nc'
filename_L3R_test = dir_out + indicative_date_time + \
    '-' + ISDP + '-' + 'L3R' + '-' + SST_type + '-' +Product_string+ '-v' +Annex_version+ '-fv' +File_version+ 'test.nc'
filename_L3R

#l3r.to_netcdf(filename_L3R)

#for some reason the time not decoded is off by about 28 seconds so reset to original decoded time here
l3r['time']=ds.time[0,:].data                 
l3r.to_netcdf(filename_L3R)


C:\Users\gentemann\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in less


In [9]:
print(WD[100:800])
#print(WD.max())
#print(l3r.wind_direction[300:400])
#l3r.wind_direction[0:100].plot()  

[ 35.74562333  38.14295988  32.86521305  33.33222538  38.3201463
  32.79284488  36.13540713  36.63654585  34.92806274  30.63259662
  33.16638271  33.96852924  33.10366991  26.37296762  22.8511755
  27.30274738  37.08861409  29.07784413  30.80528841  31.7685541
  31.30414994  31.93070907  30.31837903  31.42959216  30.02459095
  26.12603372  23.58233532  23.87670854  25.79027118  22.76939246
  16.38454273  18.99798436  13.74033195  11.86515149  15.13717843
  18.26203989  13.75079804  17.2338158   10.34165403  10.97051574
  12.17484957  11.8846961   10.0806065   13.9260005    8.88894999
  16.59065549  13.10280101   7.3057657   14.653321    14.72029692
  13.94272647  17.35855149  10.91027657  12.52055288  15.58093701
  12.78280303  13.73118891  15.32969136  15.74699316  16.51834855
  15.11820088  17.86177602  18.43496439  19.31857322  18.52974649
  16.27727246  16.9947401   14.49140803  21.14681696  23.34474776
  16.20903862  21.47680912  20.81102162  20.77998261  18.00033236
  20.43431616

In [10]:
WS_height

5